# Homework 2 part b
**Instructions:**
- Submit your code to github by the deadline.
- DO NOT change paths (-3 points).
- DO NOT submit data to github (-2 points).

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

## Problem 1: log loss (3 points)
**(a)** You have a classification problem with 3 classes: "cat", "dog", "bird". For your test observation is a "dog". Your model give you the following prediction for that observation (0.1, 0.5, 0.4). What is the accuracy? What is the log loss?

**Answer (a):**

Accurcy is 1,since the model predicts the corrrect category. 

In [2]:
logloss = -np.log(1-0.5)
print(f'Log loss: {logloss:.4f}')

Log loss: 0.6931


**(b)** Suppose that you are submitting to a Kaggle competition. You are solving a binary classification task being evaluated by log loss metric. You suspect train and test target distributions are different, you submit a constant predition of 0.3 and to the public LB and get a score of 1.01. Mean of target variable in train is 0.44. What is the mean of target variable in public part of test data? Explain how you derive the results.

**Answer (b)**

In binary classification, the log loss equation is:
$$ -\frac{1}{N}(\sum_{y_i=0}log(1-\hat{y_i})+\sum_{y_i=1}log(\hat{y_i}))$$

When all predictions are 0.3, log loss becomes:
$$ -\frac{1}{N}(\sum_{y_i=0}log(0.7)+\sum_{y_i=1}log(0.3))$$

Which means:
$$ -\frac{1}{N}(N_{y_i=0}\times log(0.7)+N_{y_i=1}\times log(0.3))=1.01$$

Let $x=\frac{N_{y_i=0}}{N}$, equations are derived:
$$log(0.7)x+log(0.3)(1-x)=-1.01$$\
$$x = \frac{-1.01-log(0.3)}{log(0.7)-log(0.3)}$$

The mean of target variable in the test data will be:
$$1-x$$

In [3]:
x = (-1.01-np.log(0.3))/(np.log(0.7)-np.log(0.3))
print(f'The mean of target variable in the test data: {1-x:.4f}')

The mean of target variable in the test data: 0.7711


**(c)** You have a set of images that are tagged with labels. Each image may be tagged with more than one label. You have a total of 4 labels. Here is the soft prediction for one of your test images (0.1, 0.9, 0.8, 0.3). What is the hard prediction if you are using a threshold of 0.7? What is the loss if your real label is (0, 1, 0, 0)?

**Answer (c)** 

The hard prediction is (0, 1, 1, 0).

Log loss: 0.5442

In [4]:
loss = (np.log(1-0.1)+np.log(0.9)+np.log(1-0.8)+np.log(1-0.3))/(-4)
print(f'Log loss: {loss:.4f}')

Log loss: 0.5442


## Problem 2: AUC (2 points)
Compute AUC score by hand with the formula explained in class for the following dataset.

In [5]:
d = pd.DataFrame({
        'prediction': [0.1, 0.5, 0.95, 0.99, 0.8, 0.4, 0.03, 0.44, 0.2],
        'y': [1, 0, 1, 1, 1, 1, 0, 0, 0]})
d

,prediction,y
0,0.10,1
1,0.50,0
2,0.95,1
3,0.99,1
4,0.80,1
5,0.40,1
6,0.03,0
7,0.44,0
8,0.20,0


In [6]:
d = d.sort_values('prediction'); d

,prediction,y
6,0.03,0
0,0.10,1
8,0.20,0
5,0.40,1
7,0.44,0
1,0.50,0
4,0.80,1
2,0.95,1
3,0.99,1


In [7]:
pairs = 4*5
wrong = 3+2
auc = 1-wrong/pairs
print(f'AUC score: {auc}')

AUC score: 0.75


## Problem 3: Regularized mean (target) encoding for Avazu competition (8 points)

For this problem you will implement a version of regularized mean encoding. We will be using the data on this Kaggle [compettion](https://www.kaggle.com/c/avazu-ctr-prediction). You can use the kaggle api (install first). 

`kaggle competitions download -c avazu-ctr-prediction`

**Instructions:**
- Split data (training) into training and validation. Take the lastest (most recent data) of training set as validation.
- Implement regularized mean encoding for the training set using pandas.
- Implement mean encoding for the validation set

In [8]:
## Split train and validation 
# get sample data first
PATH = Path("avazu-ctr-prediction")
!head -100000 $PATH/train > $PATH/train_sample.csv
!head -100000 $PATH/test > $PATH/test_sample.csv


data = pd.read_csv(PATH/"train_sample.csv")
test = pd.read_csv(PATH/"test_sample.csv")

In [9]:
def split_based_hour(data):
    """ Split data based on column hour.
    
    Use 20% of the date for validation.
    Inputs:
       data: dataframe from avazu
    Returns:
       train:
       val: 20% of the largest values of column "hour".
    """
    N = int(0.8*len(data))
    data = data.sort_values(by="hour")
    train = data[:N].copy()
    val = data[N:].copy()
    return train.reset_index(), val.reset_index()
train, val = split_based_hour(data)

### Regularized mean encoding 
Here is how you do mean encoding without regularization.

In [10]:
# Calculate a mapping: {device_type: click_mean}
mean_device_type = train.groupby('device_type').click.mean()
mean_device_type

device_type
0    0.224277
1    0.176116
4    0.069777
5    0.083333
Name: click, dtype: float64

In [11]:
# This is the global click mean
global_mean = train.click.mean()
global_mean

0.17477718471480894

In [12]:
train["device_type_mean_enc"] = train["device_type"].map(mean_device_type)
val["device_type_mean_enc"] = val["device_type"].map(mean_device_type)

In [13]:
train["device_type_mean_enc"].fillna(global_mean, inplace=True)
val["device_type_mean_enc"].fillna(global_mean, inplace=True)

In [14]:
# Print correlation
encoded_feature = val["device_type_mean_enc"].values
print(np.corrcoef(val["click"].values, encoded_feature)[0][1])

0.0530389229998215


To do mean encoding with K-fold regularization you do the following:

* Run a 5-fold split on train data where `mean_device_type` is computed on 4/5 of the data and the encoding is computed on the other 1/5.
* To compute mean encoding on the validation data use the code similar to encoding without regularization. That is compute on all the training data and apply to the validation set.

In [15]:
from sklearn.model_selection import KFold

def reg_target_encoding(train, col = "device_type", splits=5):
    """ Computes regularize mean encoding.
    Inputs:
       train: training dataframe
       
    """
    ### BEGIN SOLUTION
    kf = KFold(splits)
    for compute_index, encode_index in kf.split(train):
        mean_device_type = train.iloc[compute_index].groupby(col).click.mean()
        new_col = col + '_mean_enc'
        train.loc[encode_index, new_col] = train.loc[encode_index, col].map(mean_device_type)
        global_mean = train.iloc[compute_index].click.mean()
        train.loc[encode_index, new_col].fillna(global_mean, inplace=True)
    ### END SOLUTION

In [16]:
reg_target_encoding(train) 
encoded_feature = train["device_type_mean_enc"].values
corr = np.corrcoef(train["click"].values, encoded_feature)[0][1]
assert(np.around(corr, decimals=4) == 0.0551)

In [17]:
def mean_encoding_test(test, train, col = "device_type"):
    """ Computes target enconding for test data.
    
    This is similar to how we do validation
    """
    ### BEGIN SOLUTION
    new_col = col + '_mean_enc'
    mean_device_type = train.groupby(col).click.mean()
    global_mean = train.click.mean()

    train[new_col] = train[col].map(mean_device_type)
    test[new_col] = test[col].map(mean_device_type)

    train[new_col].fillna(global_mean, inplace=True)
    test[new_col].fillna(global_mean, inplace=True)
    ### END SOLUTION

In [18]:
mean_encoding_test(test, train) 
encoded_feature_mean = test["device_type_mean_enc"].values.mean()
assert(np.around(encoded_feature_mean, decimals=4) == 0.177)

## Problem 4: Implement other features and fit a model (7 points)
* Implement a few more features, include:
   * day of the week and hour
   * mean encoding of some other features (at least two)
   * use plots and `value_counts()` to understand the data

* Fit a random forest (to the whole dataset)
   * Do hyperparameter tunning using your validation set
   * Report validation and train log loss

In [19]:
import datetime, time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt

%matplotlib inline

In [22]:
pd.set_option('display.max_columns', 30)

In [23]:
PATH = Path("avazu-ctr-prediction")
types = {'id': np.uint32, 'click': np.uint8, 'hour': np.uint32, 'C1': np.uint32, 'banner_pos': np.uint32,
         'site_id': 'category', 'site_domain': 'category', 'site_category': 'category', 'app_id': 'category',
         'app_domain': 'category', 'app_category': 'category', 'device_id': 'category',
         'device_ip': 'category', 'device_model': 'category', 'device_type': np.uint8, 'device_conn_type': np.uint8,
         'C14': np.uint16, 'C15': np.uint16, 'C16': np.uint16, 'C17': np.uint16, 'C18': np.uint16, 'C19': np.uint16,
         'C20': np.uint16, 'C21': np.uint16}
data = pd.read_csv(PATH/"train", usecols=types.keys(), dtype=types)

In [24]:
train, val = split_based_hour(data)

### Day of the week and hour

In [25]:
def dow_hour(train):
    train['date_time'] = pd.to_datetime(train['hour'], format='%y%m%d%H', errors='raise')
    train['Hour'] = train['date_time'].dt.hour
    train['dow'] = train['date_time'].dt.dayofweek

In [26]:
dow_hour(train)
dow_hour(val)

### use plots and value_counts() to understand the data

In [27]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32343173 entries, 0 to 32343172
Data columns (total 28 columns):
index               int64
id                  uint32
click               uint8
hour                uint32
C1                  uint32
banner_pos          uint32
site_id             category
site_domain         category
site_category       category
app_id              category
app_domain          category
app_category        category
device_id           category
device_ip           category
device_model        category
device_type         uint8
device_conn_type    uint8
C14                 uint16
C15                 uint16
C16                 uint16
C17                 uint16
C18                 uint16
C19                 uint16
C20                 uint16
C21                 uint16
date_time           datetime64[ns]
Hour                int64
dow                 int64
dtypes: category(9), datetime64[ns](1), int64(3), uint16(8), uint32(4), uint8(3)
memory usage: 3.1 GB


In [28]:
train.describe()

,index,id,click,hour,C1,banner_pos,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21,Hour,dow
count,3.234317e+07,3.234317e+07,3.234317e+07,3.234317e+07,3.234317e+07,3.234317e+07,3.234317e+07,3.234317e+07,3.234317e+07,3.234317e+07,3.234317e+07,3.234317e+07,3.234317e+07,3.234317e+07,3.234317e+07,3.234317e+07,3.234317e+07,3.234317e+07
mean,1.617165e+07,2.147647e+09,1.714687e-01,1.410246e+07,1.004966e+03,2.883468e-01,1.015254e+00,3.276667e-01,1.839799e+04,3.187774e+02,6.039949e+01,2.057456e+03,1.424106e+00,2.223480e+02,4.897828e+04,8.061089e+01,1.131493e+01,2.622954e+00
std,9.336779e+06,1.239852e+09,3.769180e-01,2.412696e+02,1.101809e+00,5.046780e-01,5.310223e-01,8.550944e-01,4.997212e+03,2.170879e+01,4.793508e+01,6.104181e+02,1.322267e+00,3.513966e+02,1.544234e+04,6.608124e+01,5.908739e+00,1.914751e+00
min,0.000000e+00,2.170000e+02,0.000000e+00,1.410210e+07,1.001000e+03,0.000000e+00,0.000000e+00,0.000000e+00,3.750000e+02,1.200000e+02,2.000000e+01,1.120000e+02,0.000000e+00,3.300000e+01,3.446400e+04,1.300000e+01,0.000000e+00,0.000000e+00
25%,8.085793e+06,1.074365e+09,0.000000e+00,1.410222e+07,1.005000e+03,0.000000e+00,1.000000e+00,0.000000e+00,1.661500e+04,3.200000e+02,5.000000e+01,1.800000e+03,0.000000e+00,3.500000e+01,3.454800e+04,2.300000e+01,7.000000e+00,1.000000e+00
50%,1.617159e+07,2.147469e+09,0.000000e+00,1.410242e+07,1.005000e+03,0.000000e+00,1.000000e+00,0.000000e+00,2.010800e+04,3.200000e+02,5.000000e+01,2.281000e+03,2.000000e+00,3.900000e+01,3.465700e+04,6.100000e+01,1.100000e+01,2.000000e+00
75%,2.425738e+07,3.221479e+09,0.000000e+00,1.410270e+07,1.005000e+03,1.000000e+00,1.000000e+00,0.000000e+00,2.176600e+04,3.200000e+02,5.000000e+01,2.502000e+03,3.000000e+00,1.670000e+02,6.553500e+04,9.500000e+01,1.600000e+01,4.000000e+00
max,3.237742e+07,4.294967e+09,1.000000e+00,1.410282e+07,1.012000e+03,7.000000e+00,5.000000e+00,5.000000e+00,2.351900e+04,1.024000e+03,1.024000e+03,2.688000e+03,3.000000e+00,1.959000e+03,6.553500e+04,2.210000e+02,2.300000e+01,6.000000e+00


In [29]:
data["click"].value_counts()

0    33563901
1     6865066
Name: click, dtype: int64

In [30]:
data["banner_pos"].value_counts()

0    29109590
1    11247282
7       43577
2       13001
4        7704
5        5778
3        2035
Name: banner_pos, dtype: int64

In [31]:
data["app_category"].value_counts()

07d7df22    26165592
0f2161f8     9561058
cef3e649     1731545
8ded1f7a     1467257
f95efa07     1141673
d1327cf5      123233
09481d60       54886
dc97ec06       54644
75d80bbe       40108
fc6fa53d       23663
4ce2e9fc       20762
879c24eb       12785
a3c42688       11121
4681bb9d        6291
0f9a328c        5584
a86a3e89        2497
2281a340        2243
8df2e842        1679
79f0b860         605
0bfbc358         425
a7fd01ec         362
7113d72a         304
2fc4f2aa         234
18b1e0be         212
5326cf99         133
0d82db25          25
4b7ade46          16
bf8ac856           9
bd41f328           6
71af18ce           5
86c1a5a3           3
6fea3693           2
ef03ae90           2
52de74cf           1
f395a87f           1
cba0e20d           1
Name: app_category, dtype: int64

In [32]:
val.loc[~val.app_category.isin(train.app_category.unique())]

,index,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,device_model,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21,date_time,Hour,dow


### mean encoding of some other features (at least two)

In [33]:
# using device_type from previous
reg_target_encoding(train, col = "device_type", splits=5)
mean_encoding_test(val, train, col = "device_type")

In [34]:
# mean encoding app_category
reg_target_encoding(train, col = "app_category", splits=5)
mean_encoding_test(val, train, col = "app_category")

In [35]:
# mean encoding banner_pos
reg_target_encoding(train, col = "banner_pos", splits=5)
mean_encoding_test(val, train, col = "banner_pos")

### Fit a random forest (to the whole dataset)
   * Do hyperparameter tunning using your validation set
   * Report validation and train log loss

In [36]:
def data_array(df):
    X = df.select_dtypes(include=np.number)\
           .drop(columns=['device_conn_type', 'click', 'id', 'index'])\
           .values
    y = df['click'].values
    return X, y
X_train, y_train = data_array(train)
X_val, y_val = data_array(val)

In [40]:
rf = RandomForestClassifier(n_estimators=50, criterion="gini", n_jobs=-1)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [41]:
y_train_pred = rf.predict_proba(X_train)
train_loss = log_loss(y_train, y_train_pred)

y_val_pred = rf.predict_proba(X_val)
val_loss = log_loss(y_val, y_val_pred)
print(f'Train log loss: {train_loss:.4f}, Validation log loss: {val_loss:.4f}')

Train log loss: 0.3967, Validation log loss: 0.4195
